In [5]:
# 導入必要的套件
!pip install pycaret optuna

# Import required libraries
import pandas as pd
import seaborn as sns
from pycaret.classification import *
import optuna

# Step 1: Load Titanic dataset
def load_data():
    # Load dataset from seaborn or local file
    data = sns.load_dataset('titanic')
    return data

# Step 2: Preprocess the Titanic dataset
def preprocess_data(data):
    data = data.copy()

    # Handle missing values
    data['age'].fillna(data['age'].median(), inplace=True)
    data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)

    # Convert 'deck' to a string type before filling
    data['deck'] = data['deck'].astype(str)
    data['deck'].fillna('Unknown', inplace=True)

    # Ensure target column 'survived' is numeric
    data['survived'] = data['survived'].astype(int)

    # Encode categorical variables
    data = pd.get_dummies(data, columns=['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive', 'alone'], drop_first=True)

    # Drop unnecessary columns if they exist
    columns_to_drop = ['name', 'ticket', 'cabin']
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

    return data

# Step 3: PyCaret setup
def setup_environment(data):
    clf = setup(
        data=data,
        target='survived',
        normalize=True,
        feature_selection=True, # Disable polynomial features to avoid feature mismatch
        session_id=123,
        verbose=False
    )
    return clf

# Step 4: Model selection using compare_models
def model_selection():
    # Compare 16 models and return the best one
    best_model = compare_models()
    return best_model

# Step 5: Hyperparameter optimization with Optuna
def hyperparameter_optimization(data):
    def objective(trial):
        try:
            clf = setup(
                data=data,
                target='survived',
                normalize=True,
                session_id=123,
                verbose=False
            )
            # Suggest model and tune it
            model_name = trial.suggest_categorical("model", ['lr', 'rf', 'svm', 'xgboost'])
            model = create_model(model_name)
            tuned_model = tune_model(model, optimize='Accuracy')

            # Pull metrics and ensure accuracy exists
            metrics = pull()
            if 'Accuracy' in metrics.columns:
                return metrics['Accuracy'].iloc[0]
            elif 'accuracy' in metrics.columns:  # Handle lowercase
                return metrics['accuracy'].iloc[0]
            else:
                print("Accuracy metric not found. Returning 0.")
                return 0
        except Exception as e:
            print(f"Error during trial: {e}")
            return 0

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)

    print(f"Best hyperparameters: {study.best_params}")

# Step 6: Train and finalize the best model
def train_and_finalize_model(best_model):
    # Tune and finalize the model
    tuned_model = tune_model(best_model, optimize='Accuracy')
    final_model = finalize_model(tuned_model)
    return final_model

# Main Function
def main():
    # Load and preprocess the dataset
    data = load_data()
    preprocessed_data = preprocess_data(data)

    # Setup environment
    setup_environment(preprocessed_data)

    # Model selection
    best_model = model_selection()
    print(f"Best model selected: {best_model}")

    # Hyperparameter optimization using Optuna
    hyperparameter_optimization(preprocessed_data)

    # Train and finalize the best model
    final_model = train_and_finalize_model(best_model)

    # Save the final model
    save_model(final_model, 'best_titanic_model')

    print("Model training, tuning, and saving completed.")

if __name__ == "__main__":
    main()


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 235
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.5400
nb,Naive Bayes,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1160
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0690
svm,SVM - Linear Kernel,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0670
ridge,Ridge Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0680
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2460
qda,Quadratic Discriminant Analysis,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0730
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0730
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1430
lda,Linear Discriminant Analysis,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0790


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

[I 2024-12-17 06:24:16,622] A new study created in memory with name: no-name-b812ccab-4ef3-41b9-b0c0-d463e91aa203


Best model selected: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


...

Best hyperparameters: {'model': 'lr'}


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Transformation Pipeline and Model Successfully Saved
Model training, tuning, and saving completed.
